# Jetracer Data Collection Notebook
Author: George Gorospe

About: Data collection for the jetracer platform is here made easier through automated data collection. A simple timer is used to collect images at a controlled interval. Collected images may be placed in specified folders. 

Our goal is to manually drive our racer on a track while collecting data. Afterwards the data can be labeled with a different notebook.

In [1]:
# Importing the required libraries
# IPython Libraries for display and widgets
import traitlets
import ipywidgets
import ipywidgets.widgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display


# Camera and Motor Interface for JetBot
from jetbot import Camera, bgr8_to_jpeg

# Python basic pakcages for image annotation
from uuid import uuid1
import os
import glob
import datetime
import numpy as np
import cv2
import time

ModuleNotFoundError: No module named 'jetbot'

### Todo list, a collection of possible improvements for this notebook:
1. Add a short timer before collection begins
2. Add a 'desired number of samples to collect" control

### Creating a Folder to Hold the Collected Data
We can use python to create a folder to hold our data.
We'll use ipywidgets to create a dialog box to get a folder name from the user.
If the folder specified by the user already exists, we'll notify the user.

In [2]:
import os  # This library allows us to explore and modify the file structure on the Nano
# Creating a function to add a folder to our directory
def folder_function(label):
    # Using the "try/except" statement here because the makedirs function can throw an error if the directory exists already
    try:
        os.makedirs('data/'+label)
        print("Creating data folder: data/"+label)
    except:
        print('Directory not created because it already exists')
                
    
# Creating the interactive text box widget named 'label'
interactiveTextBox = interactive(folder_function,{'manual': True}, label = widgets.Text(value='structured_string',placeholder = 'structured_string',description='Enter Label:'));
interactiveTextBox # calling the interactive element

interactive(children=(Text(value='structured_string', description='Enter Label:', placeholder='structured_stri…

Here we want to create a new folder with our specific name. In this case the name will be representative of the full dataset inside. So if you're collecting data at a school, "school" is a good choice.
Or if you're collecting during daytime, "school_daytime" can be a good, representative folder name.

The folder name is very important in this case because the actual data file names will be automatically generated later, so we want the folder name to be very descriptive. 

We can use the interact_manual function to create a textbox which carries out the folder_function and creates our new folder.

Next we'll define the save_snapshot function that will collect an image with the camera and save it to the new folder.
Since we don't want to manually name each image we collect, we'll use the ``uuid`` package in python, which defines the ``uuid1`` method to generate
a unique identifier.  This unique identifier is generated from information like the current time and the machine address. We go through this complicated process so that we never create two files with the same name.

Next we'll create the interval duration input box. For this feature we need a min, max, and step values.
The output value is available as interval.value.

In [3]:
# Interval Duration Input Box
interval = widgets.BoundedFloatText(
    value=1.5,
    min=0.1,
    max=5.0,
    step=0.1,
    description='Interval:',
    disabled=False
)
# select your interval duration for automated data collection
display(interval)

BoundedFloatText(value=1.5, description='Interval:', max=5.0, min=0.1, step=0.1)

Setting up the RepeatedTimer:

We're now going to create a class for a RepeatedTimer object. This object will be useful in repeatedly collecting images as we drive the jetracer.

This class has a self, run, start, and stop calls.

The inputs to this class are: the interval, the function to be repeated, and any inputs to that function)

In [4]:
from threading import Timer
from uuid import uuid1

label = interactiveTextBox.children[0].value # take the label text from the interactive text box module
directory = 'data/' + label


class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False

    
def save_snapshot():
    global image_count
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path,'wb') as f:
        f.write(image.value)
        image_count.value = len(os.listdir(directory))

The next cell finishes the camera setup and user interface.

In [5]:
# Camera setup
camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

# Button creation and widget layout
button_layout = widgets.Layout(width='128px', height='64px')
# Buttons to controll data collection
start_button = widgets.Button(description='Start Collection', button_style='success', layout=button_layout)
stop_button = widgets.Button(description='Stop Collection', button_style='danger', layout=button_layout)
# image_count is used to display the number of images in the folder
image_count = widgets.IntText(layout=button_layout,continuous_update=True, value=len(os.listdir(directory)))
    
# attach the callbacks, we use a 'lambda' function to ignore the
# parameter that the on_click event would provide to our function
# because we don't need it.
start_button.on_click(lambda x: rt.start())
stop_button.on_click(lambda x: rt.stop())

# Setup of the repeated timer to handle data collection
rt = RepeatedTimer(interval.value, save_snapshot) # it auto-starts, no need of rt.start()
rt.stop()

# User Interface
display(image)
display(widgets.HBox([image_count, start_button, stop_button]))
rt.start()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…